In [2]:
import pandas as pd
import statsmodels.formula.api as smf

In [3]:
df = pd.read_csv('/Users/patrickburke/Library/CloudStorage/OneDrive-EmoryUniversity/ECON496RW/raceANDcomp/v2.csv')

In [4]:
df.head()

,PcrKey,black,white,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander,ageinyear,EMSSystemResponseTimeMin
0,13273412,0,1,0,0,0,0,78,8.633333
1,83733589,0,0,0,0,1,0,31,29.000000
2,88178354,0,1,0,0,0,0,85,3.000000
3,97455081,0,1,0,0,0,0,77,109.000000
4,102488422,0,1,0,0,0,0,4,31.000000


In [5]:
df = df.sample(n=100000)

In [6]:
df.head()

,PcrKey,black,white,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander,ageinyear,EMSSystemResponseTimeMin
2329252,124213748,0,0,1,0,0,0,21,55.966667
19422649,153052079,0,1,0,0,0,0,33,12.516667
31914408,179762618,0,1,0,0,0,0,66,24.000000
16285733,148157358,0,1,0,0,0,0,31,5.000000
18128305,151028473,0,1,0,0,0,0,72,7.000000


In [7]:
# classic OLS right here
m1 = smf.ols(formula='''EMSSystemResponseTimeMin ~ 
                        ageinyear +
                        black +
                        white +
                        hispanic_latino +
                    	asian +
                        americanIndian_alaskaNative +
                        nativeHawaiian_otherPacificIslander	
                     ''', data=df).fit()

In [8]:
m1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     EMSSystemResponseTimeMin   R-squared:                       0.005
Model:                                  OLS   Adj. R-squared:                  0.005
Method:                       Least Squares   F-statistic:                     66.90
Date:                      Fri, 16 Sep 2022   Prob (F-statistic):           8.97e-97
Time:                              04:39:58   Log-Likelihood:            -5.3747e+05
No. Observations:                    100000   AIC:                         1.075e+06
Df Residuals:                         99992   BIC:                         1.075e+06
Df Model:                                 7                                         
Covariance Type:                  nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               9.2082      1.182      7.793      0.000       6.892      11.524
ageinyear                               0.1287      0.007     17.410      0.000       0.114       0.143
black                                   4.3625      1.127      3.872      0.000       2.154       6.571
white                                  -1.7686      1.094     -1.617      0.106      -3.912       0.375
hispanic_latino                        -0.0229      1.141     -0.020      0.984      -2.259       2.213
asian                                   2.3857      1.620      1.472      0.141      -0.790       5.561
americanIndian_alaskaNative             0.9865      1.812      0.544      0.586      -2.565       4.538
nativeHawaiian_otherPacificIslander    -3.9014      2.738     -1.425      0.154      -9.267       1.464
==============================================================================
Omnibus:                   195816.650   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        429506971.575
Skew:                          15.843   Prob(JB):                         0.00
Kurtosis:                     322.496   Cond. No.                     1.06e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.06e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""